Đọc dữ liệu đã làm sạch


In [1]:
import pandas as pd
from pathlib import Path

In [2]:
def find_csv(name):
    for base in [Path.cwd()] + list(Path.cwd().parents):
        files = list(base.rglob(name))
        if files:
            return files[0]
    raise FileNotFoundError(f"Không tìm thấy file {name}")

info  = pd.read_csv(find_csv("stock_info_clean.csv"))
price = pd.read_csv(find_csv("stock_price_clean.csv"), parse_dates=["date"])
trans = pd.read_csv(find_csv("transactions_clean.csv"))


giá đóng cửa trung bình theo ngày


In [3]:
# Gộp bảng giá với thông tin ngành
df = price.merge(info, on="ticker", how="left")

if 'sector' in df.columns and 'close_price' in df.columns:
    avg_price_sector = df.groupby("sector")["close_price"].mean()

    print("Giá đóng cửa trung bình theo ngành:")
    print(avg_price_sector.sort_values(ascending=False))


Giá đóng cửa trung bình theo ngành:
sector
Tài Chính    42311.000
Bán Lẻ       40312.375
Tai Chinh    40062.375
Name: close_price, dtype: float64


Khối lượng giao dịch trung bình theo sàn


In [4]:
# Gộp transaction với thông tin sàn
trans_info = trans.merge(info, on="ticker", how="left")
if 'exchange' in trans_info.columns and 'volume' in trans_info.columns:
    avg_volume_exchange = trans_info.groupby("exchange")["volume"].mean()

    print("\nKhối lượng giao dịch trung bình theo sàn:")
    print(avg_volume_exchange)



Khối lượng giao dịch trung bình theo sàn:
exchange
HOSE     11001.50
UPCOM     9252.75
Name: volume, dtype: float64


Mã có biến động giá mạnh nhất

In [5]:
if 'ticker' in df.columns and 'close_price' in df.columns:
    price_volatility = df.groupby("ticker")["close_price"].std()

    most_volatile = price_volatility.sort_values(ascending=False).head(1)

    print("\nMã có biến động giá mạnh nhất:")
    print(most_volatile)


Mã có biến động giá mạnh nhất:
ticker
FPT    25195.585454
Name: close_price, dtype: float64


Tổng hợp dữ liệu theo tháng

In [6]:
# Chuyển cột date sang datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Tạo cột tháng
if 'date' in df.columns:
    df["month"] = df["date"].dt.to_period("M").astype(str)

# Giá trung bình theo tháng
monthly_price = df.groupby("month")["close_price"].mean()

print("\nGiá đóng cửa trung bình theo tháng:")
print(monthly_price)



Giá đóng cửa trung bình theo tháng:
month
2024-01    40347.083333
2024-02    30522.916667
2024-03    45239.230769
2024-04    42675.833333
2024-05    47117.222222
2024-06    29022.916667
2024-07    47783.888889
2024-08    41123.333333
2024-09    47006.875000
2024-10    45342.500000
2024-11    25027.500000
2024-12    36685.000000
NaT        41908.184932
Name: close_price, dtype: float64


Ngành có thanh khoản thấp nhất (Khối lượng giao dịch trung bình thấp nhất)

In [7]:
# Gộp transaction với ngành
trans_sector = trans.merge(info, on="ticker", how="left")
if 'sector' in trans_sector.columns and 'volume' in trans_sector.columns:
    liquidity_sector = trans_sector.groupby("sector")["volume"].mean()

    lowest_liquidity = liquidity_sector.sort_values().head(1)

    print("\nNgành có thanh khoản thấp nhất:")
    print(lowest_liquidity)



Ngành có thanh khoản thấp nhất:
sector
Bán Lẻ    8753.0
Name: volume, dtype: float64
